## Lab01: Thu thập dữ liệu

- Họ tên: Nguyễn Minh Tuấn
- MSSV: 20127092

Check môi trường code:

In [1]:
import sys
sys.executable

'/home/minhtuan2511/anaconda3/envs/min_ds-env/bin/python'

Nếu không có vấn đề gì thì file chạy python sẽ là file của môi trường code "min_ds-env".

***
## Import

In [9]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian
# YOUR CODE HERE (OPTION) 
# Nếu cần các thư viện khác thì bạn có thể import ở đây
from requests import Response
import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không

---

## Setup ở mức toàn cục (1đ)

In [10]:
sleep_time = 1 # Đơn vị: giây

In [11]:
# YOUR CODE HERE
def check(response: Response) -> bool:
    return '"incomplete_results":true' not in response.text

requests_cache.install_cache(filter_fn = check)   
    

In [12]:
# TEST
url1 = 'https://raw.githubusercontent.com/KienTrann/requests-cache-testing/main/should_be_cached.txt'
r = requests.get(url1) # Thực hiện request lần 1
r = requests.get(url1) # Thực hiện request lần 2
print(f'Text from url1:\n{r.text}')
assert r.from_cache == True


url2 = 'https://raw.githubusercontent.com/KienTrann/requests-cache-testing/main/should_not_be_cached.txt'
r = requests.get(url2) # Thực hiện request lần 1
r = requests.get(url2) # Thực hiện request lần 2
print('---')
print(f'Text from url2:\n{r.text}')
assert r.from_cache == False


Text from url1:
abc
xyz
"incomplete_results":false

---
Text from url2:
abc
xyz
"incomplete_results":true



---

## Thu thập dữ liệu từ web bằng cách parse HTML (4đ)

[Coursera](https://coursera.org) là trang cung cấp các khóa học online, là một nguồn tài liệu hữu ích để học tập. Trong phần này, bạn sẽ thu thập dữ liệu về các khóa học ở Coursera về một chủ đề nào đó. Mình có check ["Terms of Use" của Coursera](https://www.coursera.org/about/terms) thì không thấy nói rõ là cấm parse HTML (bạn check lại thử xem); với mục đích học thì chắc là không có vấn đề gì lớn, miễn là chương trình của bạn đừng "hit" trang web quá nhiều lần trong một khoảng thời gian ngắn. 

Công việc cụ thể của bạn trong phần này là viết hàm `collect_courses` ở bên dưới. Hàm này có các input như sau:
- `course_urls_file`: là chuỗi, cho biết tên của file txt chứa các đường link đến tất cả khóa học của Coursera, mỗi đường link nằm trên một dòng; bạn sẽ đi theo đường link để lấy thông tin của khóa học. File này đã được cung cấp cho bạn: file "course_urls.txt" (mình tạo ra file này bằng cách parse [trang XML này](https://www.coursera.org/sitemap~www~courses.xml)).
- `key_words`: là list, cho biết các từ khóa để lọc ra các link cần lấy. Ví dụ: `key_words=['data-science', 'datascience']` sẽ chỉ thu thập thông tin từ các link mà có từ "data-science" HOẶC "datascience" (có ở tên đường link, chẳng hạn: https://www.coursera.org/learn/spatial-data-science).
- `courses_file`: là chuỗi, cho biết tên của file tsv dùng để lưu dữ liệu thu thập được; đây là file output (file này không có sẵn mà sẽ được tạo ra sau khi chạy hàm của bạn). Mình có cung cấp file output đúng "correct_courses.tsv" (khi gọi hàm `collect_courses` với `key_words=['data-science', 'datascience']`) để bạn hình dung được nội dung cần có của file output. File này có các phần tử trên mỗi dòng được phân tách nhau bởi tab (`\t`), và file này mã hóa utf-8. File này gồm có các cột:
    - `url`: đường link đến khóa học.
    - `title`: tựa đề của khóa học.
    - `provider`: trường cung cấp khóa học.
    - `rating`: điểm số của khóa học (tại thời điểm bạn thu thập dữ liệu thì giá trị của cột này có thể sẽ hơi khác so với file output đúng của mình, trong trường hợp chưa có ai rate khóa học thì bạn có thể cho `rating` bằng `None`).
    - `num_ratings`: số lượng người đã cho điểm cho khóa học (tại thời điểm bạn thu thập dữ liệu thì giá trị của cột này có thể sẽ hơi khác so với file output đúng của mình, trong trường hợp chưa có ai rate khóa học thì bạn có thể cho `num_rating` bằng `None`).

Trong hàm này, khi thực hiện một request, nếu kết quả không được lấy từ cache thì bạn cần cho chương trình sleep rồi mới tiếp tục.

In [13]:
# Xem 5 dòng đầu của file "correct_courses.tsv"
df = pd.read_csv('correct_courses.tsv', sep='\t')
df.head()

,url,title,provider,rating,num_ratings
0,https://www.coursera.org/learn/advanced-clinic...,Advanced Clinical Data Science,University of Colorado System,None,None
1,https://www.coursera.org/learn/advanced-data-s...,Advanced Data Science Capstone,IBM,4.6,360
2,https://www.coursera.org/learn/applied-data-sc...,Applied Data Science Capstone,IBM,4.7,"5,549"
3,https://www.coursera.org/learn/applied-data-sc...,كابستون علوم البيانات التطبيقية,IBM,None,None
4,https://www.coursera.org/learn/applied-data-sc...,Applied Data Science for Data Analysts,Databricks,4.2,24


In [14]:
def identifyBrokenLinks(uniqueExternalLinks):
    count = 0
    length_uniqueExternalLinks = len(uniqueExternalLinks)
    user_agent = {'User-Agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'}
    brokenLinksList = []
    for link in uniqueExternalLinks:     
        count = count + 1       
        try:    
            statusCode = requests.get(link, headers=user_agent).status_code        
            if statusCode == 404:         
                brokenLinksList.append(link)         
            else:        
                pass    
        except: 
            brokenLinksList.append(link)
    return brokenLinksList

def collect_courses(course_urls_file, key_words, courses_file):
    with open(course_urls_file) as f:
        lines = f.readlines()
    
    titles = []
    
    for i in lines:
        for key in key_words:
            if i.find(key) != -1:
                i = i.replace('\n','')
                titles.append(i)
                
    brokenlink = identifyBrokenLinks(titles)
    for link in brokenlink:
        if link in titles:
            titles.remove(link)
            
    f.close()
    
    file = open(courses_file, 'w+')
    file.write('url\ttitle\tprovider\trating\tnum_ratings\n')
    for i in range(len(titles)):
        file.write(titles[i])
        file.write('\t')
        
        r = requests.get(titles[i])
        temp = BeautifulSoup(r.content, 'html.parser')
        
        coursename = temp.find('h1', class_ = 'banner-title banner-title-without--subtitle m-b-0').text
        file.write(coursename)
        file.write('\t')
        
        provider = temp.find('h3', class_ = 'headline-4-text bold rc-Partner__title').text
        file.write(provider)
        file.write('\t')
        
        rating = temp.find('span', class_ = '_16ni8zai m-b-0 rating-text number-rating m-l-1s m-r-1')
        if rating == None:
            file.write('None')
          
        else:
            rating = temp.find('span', class_ = '_16ni8zai m-b-0 rating-text number-rating m-l-1s m-r-1').text
            rating = rating.replace('stars','')
            file.write(rating)
        file.write('\t')
        
        numrating = temp.find('span', attrs = {'data-test':'ratings-count-without-asterisks'})
        if numrating == None:
            file.write('None')
        else:
            numrating = temp.find('span', attrs = {'data-test':'ratings-count-without-asterisks'}).text
            numrating = numrating.replace(' ratings','')
            file.write(numrating)
        file.write('\n')    
        
        
    file.close()

In [16]:
# TEST (bằng mắt)
courses

,url,title,provider,rating,num_ratings
0,https://www.coursera.org/learn/advanced-clinic...,Advanced Clinical Data Science,University of Colorado System,4.8,17
1,https://www.coursera.org/learn/advanced-data-s...,Advanced Data Science Capstone,IBM Skills Network,4.6,374
2,https://www.coursera.org/learn/applied-data-sc...,Applied Data Science Capstone,IBM Skills Network,4.7,"6,310"
3,https://www.coursera.org/learn/applied-data-sc...,كابستون علوم البيانات التطبيقية,IBM Skills Network,None,None
4,https://www.coursera.org/learn/applied-data-sc...,Applied Data Science for Data Analysts,Databricks,4.3,31
5,https://www.coursera.org/learn/build-data-scie...,Building a Data Science Team,Johns Hopkins University,4.5,"3,240"
6,https://www.coursera.org/learn/code-free-data-...,Code Free Data Science,University of California San Diego,4.3,180
7,https://www.coursera.org/learn/cybersecurity-f...,Cybersecurity for Data Science,University of Colorado Boulder,4.1,10
8,https://www.coursera.org/learn/data-science-as...,Data Science as a Field,University of Colorado Boulder,4.4,17
9,https://www.coursera.org/learn/data-science-co...,A Crash Course in Data Science,Johns Hopkins University,4.5,"8,009"


***
## Thu thập dữ liệu từ web bằng cách dùng web API

### Thu thập ít hơn hoặc bằng 1000 thùng chứa (2.5đ)

Đầu tiên, bạn sẽ viết hàm `collect_repositories` ở bên dưới. Hàm này có các input:
- `start_date` và `end_date`: là chuỗi, cho biết là muốn tìm các thùng chứa được tạo từ ngày nào đến ngày nào (bao gồm cả 2 đầu). Ví dụ, `start_date='2017-01-01'` (năm-tháng-ngày) và `end_date='2017-01-31'`.
- `keyword`: là chuỗi, cho biết là muốn tìm kiếm các thùng chứa với từ khóa nào (xuất hiện ở tên thùng chứa hoặc phần mô tả của thùng chứa). Ví dụ, `keyword='data science'`.
- `per_page`: là số nguyên, cho biết là muốn bao nhiêu kết quả trên một page. Hiện giờ GitHub cho tối đa là `per_page=100`.

Trong hàm này bạn sẽ vào đường link `f'https://api.github.com/search/repositories?q={keyword} created:{start_date}..{end_date}&per_page={per_page}'` (dạng f-string trong Python) để thu thập dữ liệu. Ví dụ, bạn có thể vào thử ở web browser: https://api.github.com/search/repositories?q=data%20science%20created:2017-01-01..2017-01-31&per_page=100 (khi nhập một đường link vào web browser và enter thì một số ký tự sẽ bị mã hóa; ví dụ khoảng trắng sẽ bị mã hóa là %20 hoặc +), hoặc dùng thư viện Requests:

In [17]:
# Thư viện requests sẽ tự động mã hóa khoảng trắng cho bạn
# Bạn có thể xem đường link sau khi đã mã hóa bằng câu lệnh: r.url
r = requests.get('https://api.github.com/search/repositories?q=data science created:2017-01-01..2017-01-31&per_page=100')

In [18]:
# Xem 100 ký tự đầu tiên trong chuỗi JSON lấy được
print(r.text[:100])

{"total_count":1525,"incomplete_results":false,"items":[{"id":79030157,"node_id":"MDEwOlJlcG9zaXRvcn


Bạn thấy "total_count" có giá trị là 1546, nghĩa là tìm được 1546 thùng chứa (tại thời điểm bạn chạy thì có thể khác một ít, vì có thể xảy ra trường hợp chủ thùng chứa xóa thùng chứa đi, hoặc chuyển sang private). Tuy nhiên, đây chỉ là trang đầu tiên và chỉ có 100 thùng chứa (số lượng phần tử của "items"). Link tới trang kế tiếp nằm trong headers của nội dung gửi về từ server.

In [19]:
r.headers['Link']

'<https://api.github.com/search/repositories?q=data+science+created%3A2017-01-01..2017-01-31&per_page=100&page=2>; rel="next", <https://api.github.com/search/repositories?q=data+science+created%3A2017-01-01..2017-01-31&per_page=100&page=10>; rel="last"'

Link tới trang kế tiếp là link mà ở bên cạnh phía phải có từ `rel="next"`. `rel="last"` nghĩa là trang cuối. Vì đây là trang đầu tiên nên chỉ có "next" và "last", trang ở giữa còn có thêm "first" (trang đầu) và "prev" (trang trước). Bạn có thể xem thêm ở [document](https://docs.github.com/en/free-pro-team@latest/rest/guides/traversing-with-pagination).

Nếu để ý thì bạn thấy trang cuối là `page=10`, nghĩa là tất cả các trang chỉ có $10\times100=1000$ thùng chứa. Trong khi đó, "total_count" là 1546. Lý do: "the GitHub Search API provides up to **1,000 results for each search**" ([document](https://docs.github.com/en/free-pro-team@latest/rest/reference/search)). Trong hàm này chỉ yêu cầu bạn lấy hết kết quả ở tất cả các trang là được ($\le1000$ kết quả). Việc lấy $>1000$ kết quả sẽ làm ở phần kế tiếp.

Một yêu cầu nữa trong hàm này là: 
- Khi request không được (`r.ok` có giá trị là `False`) do hết số lượng request được phép trong 1 phút (với API không chứng thực thì hiện nay được 10 request / phút) thì bạn sẽ cho chương trình sleep rồi dậy thực hiện request lại, nếu vẫn chưa được thì lại sleep ... Bạn lưu ý là, <font color=red>không được giả định lần request đầu tiên là chắc chắn thành công</font> (vì khi chấm bài mình sẽ phải chạy nhiều bài và có thể đến bài của bạn đã hết lượt request được phép).
- Khi request được nhưng giá trị của `incomplete_results`(xem thêm ở [document](https://docs.github.com/en/free-pro-team@latest/rest/reference/search#timeouts-and-incomplete-results)) bằng `True` thì bạn cần thực hiện request lại cho đến khi `incomplete_results` bằng `False`. Bạn còn nhớ khi setup cache ở đầu, mình yêu cầu bạn không cache lại các response mà phần text có chứa chuỗi `"incomplete_results":True`. Vì nếu bạn cache lại các response này thì ở lần request kế sẽ được lấy từ cache và `incomplete_results` luôn bằng `True`.

Output trả về của hàm `collect_repositories`:
- `repositories`: là list các thùng chứa lấy được (list gồm "item" của tất cả các trang).

In [20]:
def collect_repositories(start_date, end_date, keyword='data science', per_page=100):
    repositories = []
    url = f'https://api.github.com/search/repositories?q={keyword} created:{start_date}..{end_date}&per_page={per_page}'
    page = 0

    while True:   
        while True:
            r = requests.get(url)
            response = json.loads(r.text)

            if r.ok == False:
                time.sleep(sleep_time)
            elif response['incomplete_results'] == True:
                continue
            else:
                break
                
        items = response['items']       
        for i in items:
            repositories.append(i)        
        if not ('next' in r.links):
            break
        url = r.links['next']['url']
        
    return repositories 


In [21]:
# TEST
start_date = '2017-01-01'
end_date = '2017-01-15'
keyword = 'data science'
per_page = 100
repositories = collect_repositories(start_date, end_date, keyword, per_page)
print(len(repositories))

url = f'https://api.github.com/search/repositories?q={keyword} created:{start_date}..{end_date}'
while True:
    json_str = requests.get(url).text
    if '"incomplete_results":false' in json_str:
        i = json_str.find(':')
        j = json_str.find(',')
        total_count = int(json_str[i+1:j])
        break
        
assert len(repositories) == total_count 


656


### Thu thập nhiều hơn 1000 thùng chứa (2.5đ)

Để thu thập hết 1546 thùng chứa trong ví dụ ở trên, ta sẽ: 
- Chia khoảng thời gian các thùng chứa được tạo mà ta cần tìm kiếm ra các khoảng thời gian nhỏ hơn. Ví dụ chia 2017-01-01..2017-01-15 thành 2017-01-01..2017-01-07, 2017-01-08..2017-01-14, 2017-01-15..2017-01-15; ở đây, mỗi khoảng thời gian nhỏ gồm 7 ngày (kể cả 2 đầu), khoảng thời gian nhỏ sau cùng có thể sẽ không đủ 7 ngày (do bị lẻ). Trong Python, để làm việc với dữ liệu thời gian, bạn có thể dùng thư viện `datetime`; bạn có thể tự tìm hiểu về thư viện này ở: [document](https://docs.python.org/3.7/library/datetime.html), [Corey's video](https://www.youtube.com/watch?v=eirjjyP2qcQ), Google.
- Tìm kiếm với khoảng thời gian nhỏ hơn này bằng hàm `collect_repositories` ở trên (với khoảng thời gian đủ nhỏ sẽ có $\le1000$ thùng chứa).
- Kết hợp kết quả từ các lần tìm kiếm với các khoảng thời gian nhỏ này lại.

Bạn sẽ cài đặt các bước này ở hàm `collect_all_repositories` dưới đây. Hàm này có các input:
- `start_date`, `end_date`, `keyword`, `per_page`: các input này giống như ở hàm `collect_repositories`.
- `step`: là số nguyên, cho biết khoảng thời gian nhỏ gồm bao nhiêu ngày (tính cả 2 đầu). Trong ví dụ ở trên, `step=7`.

Output trả về của hàm này:
- `all_repositories`: là list tất cả các thùng chứa ("item") lấy được.

In [22]:
def collect_all_repositories(start_date, end_date, step, keyword='data science', per_page=100):
    all_repositories = []
    start_date, end_date = datetime.strptime(start_date, '%Y-%m-%d'), datetime.strptime(end_date, '%Y-%m-%d') # datetime objs
    time = timedelta(days= step - 1)
    check = 0

    while(check == 0):
        start, end = start_date, start_date + time 
        if end >= end_date:
            end = end_date
            check = 1
        
        temp = collect_repositories(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'), keyword, per_page)
        for i in temp:
            all_repositories.append(i)
        start_date = end + timedelta(days= 1)

    return all_repositories

In [23]:
# TEST
start_date = '2017-01-01'
end_date = '2017-01-31'
step = 10
keyword = 'data science'
per_page = 100
all_repositories = collect_all_repositories(start_date, end_date, step, keyword, per_page)
print(len(all_repositories))

url = f'https://api.github.com/search/repositories?q={keyword} created:{start_date}..{end_date}'
while True:
    json_str = requests.get(url).text
    if '"incomplete_results":false' in json_str:
        i = json_str.find(':')
        j = json_str.find(',')
        total_count = int(json_str[i+1:j])
        break
assert len(all_repositories) == total_count 


1525
